# Importing the libraries

In [ ]:
import random
import numpy
import pandas as pd
import multiprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import argparse
import hashlib
import os
%matplotlib inline
from pandas import DataFrame
from sklearn.linear_model import LogisticRegression

# Loading the dataset and encoding the string attributes

In [ ]:

df = pd.read_csv('./class-Neo4j.csv')
print(df.shape)
X= df.iloc[:,0:97]
y = df.iloc[:,-1]

#CSV FIlE CONVERSION
from sklearn.preprocessing import LabelEncoder
X_raw = df[['Hash','LongName']]
X = pd.DataFrame()
for col in X_raw:
    le = LabelEncoder()
    X[col] = le.fit_transform(X_raw[col].astype(str))

new_X=df.iloc[:,2:97]
data_new = pd.concat([X, new_X], ignore_index=False, sort=False, axis=1)
# data_new.fillna(0, inplace=True)

data_new = data_new.iloc[:,0:97]

for column in data_new.columns:
    maxval = data_new[column].max()
    minval = data_new[column].min()    
    if(maxval == minval):
        data_new[column] = (data_new[column] - data_new[column].min()) / (data_new[column].max() - data_new[column].min()+ .000001)        
    else:
        data_new[column] = (data_new[column] - data_new[column].min()) / (data_new[column].max() - data_new[column].min())

data = pd.concat([data_new, y], ignore_index=False, sort=False, axis=1) 
data['Bugs'] = data['Bugs'].apply(lambda x : 1 if x != 0 else x)

# Genetic Algorithm

In [ ]:
import numpy

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    crossover_point = numpy.uint8(offspring_size[1]/2)
    for k in range(offspring_size[0]):
        parent1_idx = k%parents.shape[0]
        parent2_idx = (k+1)%parents.shape[0]
        offspring[k, 0:crossover_point] = parents.iloc[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents.iloc[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover, num_mutations=1):
    mutations_counter = numpy.uint8(offspring_crossover.shape[1] / num_mutations)
    print('\n')
    for idx in range(offspring_crossover.shape[0]):
        gene_idx = mutations_counter - 1
        for mutation_num in range(num_mutations):
            random_value = numpy.random.uniform(-1.0, 1.0, 1)
            offspring_crossover[idx, gene_idx] = offspring_crossover[idx, gene_idx] + random_value
            gene_idx = gene_idx + mutations_counter
    return offspring_crossover


def select_mating_pool(population, highest_point, num_parents):
    parents = numpy.empty((num_parents, new_population.shape[1]))
    for parent_num in range(num_parents):

        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = new_population.iloc[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999

    print('parents: ')
    print(parents)
    return parents

### Fitness Function

In [ ]:
# BLOCK STARTS
#Inputs
num_weights = 97
population = data.sample(frac=.1) 
# print(population)
member_in_pop = population.shape[0]
print(member_in_pop)
pop_size = (member_in_pop, num_weights)

num_parents_mating = 30


new_population = population.iloc[:,0:97]
num_generations = 100
#BLOCK ENDS

In [ ]:
import math
import math
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
import sklearn.metrics as metrics
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier


# BLOCK STARTS
#This function is for calculating distance
def distance(p0, p1):
    """ Calculate the distance between two hydrant """
    dist = math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2)
    return dist
# BLOCK ENDS



pop_multi = numpy.random.uniform(low=0, high=1.0, size=pop_size)
dataframe_new= DataFrame(pop_multi)
# print(pop_multi)

def cal_pop_fitness(dataframe_new,population):
    pop_multi = pd.DataFrame(dataframe_new).to_numpy()
    print('\n')
    fitness = []
    highest_point = []
    high_fit = 0
    weight = []
    temp_population = DataFrame()
    j=0
    parent = numpy.ones((1, num_weights))
    temp_fit = []
    index = []
    data_new = []
    tsnelist = []
    tsne_df = []
    res = zip()
    
    for i in range(member_in_pop):
        j=j+1
#         print(i)
        #pop
        x= population.iloc[:,0:97].multiply(pop_multi[i], axis=1)
        y=population.iloc[:,-1]
        data = pd.concat([x, y], ignore_index=False, sort=False, axis=1)
        data_new.append(data)
        from sklearn.manifold import TSNE
        tsne = TSNE(n_components=2, random_state=0, perplexity=7.0)
        tsne_obj= tsne.fit_transform(data)
        tsne_df1 = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                           'bug':population.iloc[:,-1]})
        tsnelist.append(tsne_df1)
        
        X=tsne_df1.drop('bug', axis=1)
        y=tsne_df1['bug']

        forest = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, tol=1e-5))
        cv_results = cross_val_score(forest,X,y,cv=10)
        forest.fit(X, y)

        prediction = cross_val_predict(forest,X,y,cv=10)

        accuracy = metrics.accuracy_score(y, prediction)  
        precision = precision_score(y, prediction)
        recall = recall_score(y, prediction, average='binary')
 
        temp_fit.append(recall)
        index.append(i)

    res = zip(temp_fit,index)
    result = list(res)
    
    if(high_fit<max(temp_fit)):
        high_fit = max(temp_fit)
        fitness=temp_fit
        
    for i in range(member_in_pop):     
        if(high_fit == result[i][0]):
            j = result[i][1]
            temp_population = data_new[j]
            tsne_df = tsnelist[j]
            weight = pop_multi[j]
    
    
    ind = np.argpartition(temp_fit, -num_parents_mating)[-num_parents_mating:]
    
#     print('ind: ', ind)
    for i in ind:
        j = result[i][1]
        parent = np.vstack([parent, pop_multi[j]])
    
     
    parents = pd.DataFrame(parent)
    
    
    return fitness, parents, tsne_df,weight, temp_population

# Implementation

In [ ]:
import numpy
import random
from pandas import DataFrame

# print(new_population)

best_outputs = []
weight_list =[]
tsne_df = []
temp_pop = []
fit = []
# fig, ax = plt.subplots( num_generations,1,figsize=(40,50))

for generation in range(num_generations):
    print("Generation : ", generation)
    
    fitness, parent, tsne,weight, temp_population= cal_pop_fitness(dataframe_new,population)
    highest_point = parent
    weight_list.append(weight)
    tsne_df.append(tsne)
    temp_pop.append(temp_population)
    fit.append(fitness)

    offspring_crossover = crossover(highest_point,
                                       offspring_size=(abs(pop_size[0]-highest_point.shape[0]), num_weights))

    offspring_mutation = mutation(offspring_crossover, num_mutations=2)

   
    dataframe_new.iloc[0:highest_point.shape[0], :] = highest_point.values
    dataframe_new.iloc[highest_point.shape[0]:, :] = offspring_mutation
    

In [ ]:
# Important features
from sklearn.ensemble import RandomForestClassifier
d = temp_pop[0]
print(d)
X = d.iloc[:, 0:97]
y = d['Bugs']
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X,y)

feature_imp = pd.Series(forest.feature_importances_,index=data.iloc[:,0:97].columns).sort_values(ascending=False)
print(feature_imp)


In [ ]:
#plotting

feature = DataFrame(feature_imp)
temp_list = []
feature_cols = list(feature.iloc[0:30].index)
for i in range(num_generations):
    temp = temp_pop[i]
    bug = temp['Bugs']
    
    droplist = [i for i in temp.columns if i not in feature_cols]
    temp_new = temp.drop(droplist,axis=1)
    
#     new.dropna(axis=1,inplace=True)
#     new = temp_new
    for column in temp_new.columns:
        temp_new[column] = ((temp_new[column] - temp_new[column].min()) / (temp_new[column].max() - temp_new[column].min()))

    temp_new.fillna(0.0, inplace = True)

    temp_new=temp_new.applymap(lambda x: x + np.random.rand()/20.0)

    new = pd.concat([temp_new, bug],ignore_index=False, sort=False, axis=1) 
    
    plt.figure(figsize=(10, 5))
    plt.title("Plot Number = %.i" % i, fontsize=15)
    sns.scatterplot(x="X", y="Y",hue="bug",data=tsne_df[i])
    
    fig, ax = plt.subplots(figsize=(15, 6))
    pd.plotting.parallel_coordinates(
        new, 'Bugs', color=('blue', 'red'), alpha=0.5, axvlines=False
    )
    
    plt.xticks(rotation=70)
    plt.xlabel("Feature Names")
    plt.ylabel("Values")
    plt.tight_layout()
    plt.title("Finding the Feature Values of Each Data Point")
    L=plt.legend()
    L.get_texts()[0].set_text('Non-Bug')
    L.get_texts()[1].set_text('Bug')

In [ ]:
import random
new = temp_pop[28]
feature_cols = list(feature.iloc[0:30].index)
temp = temp_pop[i]
bug = temp['Bugs']
    
droplist = [i for i in temp.columns if i not in feature_cols]
temp_new = temp.drop(droplist,axis=1)


for column in temp_new.columns:
        temp_new[column] = ((temp_new[column] - temp_new[column].min()) / (temp_new[column].max() - temp_new[column].min()))

temp_new.fillna(0.0, inplace = True)

temp_new=temp_new.applymap(lambda x: x + np.random.rand()/20.0)

new = pd.concat([temp_new, bug],ignore_index=False, sort=False, axis=1) 

fig, ax = plt.subplots(figsize=(20, 6))
pd.plotting.parallel_coordinates(
        new, 'Bugs', color=('blue', 'red'), alpha=0.5, axvlines=False
    )
# ax.get_legend().remove()


In [ ]:
#fitness
avg = []
for i in range(num_generations):
    av = sum(fit[i])/len(fit[i])
    avg.append(av)

print(avg)
itr = list(range(num_generations))
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(avg,itr)

In [ ]:
#fitness
avg = []
for i in range(num_generations):
    av = max(fit[i])
    avg.append(av)

print(avg)
itr = list(range(num_generations))
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(avg,itr)

# Applying Classifiers

In [ ]:
# Import needed packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import accuracy_score
%matplotlib inline

# df = weighted_data

In [ ]:
# df['contains_bug'] = df['contains_bug'].apply(lambda x : 1 if x != 0 else x)
X = data.iloc[:, 0:97]
y = data.iloc[:, -1]
# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=1,stratify = y)


# Random Forest Classifier

## Original Dataset

In [ ]:
forest = RandomForestClassifier(n_estimators=10)
cv_results = cross_val_score(forest,X,y,cv=10)
forest.fit(X, y)

prediction = cross_val_predict(forest,X,y,cv=10)

accuracy = metrics.accuracy_score(y, prediction)
print('Accuracy: ' ,accuracy)

print(confusion_matrix(y, prediction))
precision = precision_score(y, prediction)
print('Precision: %.3f' % precision)
recall = recall_score(y, prediction, average='binary')
print('Recall: %.3f' % recall)
print('\n')

## Transformed Dataset

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

recall_val = []
for i in range(num_generations):
    print('Plot number: ', i)
    new_weighted_data = data.iloc[:,0:97].multiply(weight_list[i], axis=1)
    df2 = new_weighted_data
    y = data.iloc[:,-1]
    X = new_weighted_data

    forest2 = RandomForestClassifier(n_estimators=10)
    cv_results = cross_val_score(forest2,X,y,cv=10)
    forest2.fit(X, y)

    prediction = cross_val_predict(forest2,X,y,cv=10)

    accuracy = metrics.accuracy_score(y, prediction)
    print('Accuracy: ' ,accuracy)

    print(confusion_matrix(y, prediction))
    precision = precision_score(y, prediction)
    print('Precision: %.3f' % precision)
    recall = recall_score(y, prediction, average='binary')
    print('Recall: %.3f' % recall)
    recall_val.append(recall)
    print('\n')


In [ ]:
itr = list(range(num_generations))
zip_iterator = zip(itr, recall_val)
a_dictionary = dict(zip_iterator)
# print(a_dictionary)
res_dict = dict(sorted(a_dictionary.items(), key=lambda item: item[1], reverse = True))

## Sorted Result

In [ ]:
res_dict

In [ ]:
df_weights = DataFrame(weight_list)
df_weights.to_csv (r'softset_neo4j_weights2.csv', index = False, header=True)
w = df_weights.iloc[57, 0:97]
w = list(w)

# K Neighbors Classifier

## Original Dataset

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

X = data.iloc[:, 0:97]
y = data.iloc[:, -1]

forest = KNeighborsClassifier()
cv_results = cross_val_score(forest,X,y,cv=10)
forest.fit(X, y)

prediction = cross_val_predict(forest,X,y,cv=10)

accuracy = metrics.accuracy_score(y, prediction)
print('Accuracy: ' ,accuracy)

print(confusion_matrix(y, prediction))
precision = precision_score(y, prediction)
print('Precision: %.3f' % precision)
recall = recall_score(y, prediction, average='binary')
print('Recall: %.3f' % recall)
print('\n')

## Transformed Dataset

In [ ]:
recall_val = []
for i in range(num_generations):
    print('Plot number: ', i)
    new_weighted_data = data.iloc[:,0:97].multiply(weight_list[i], axis=1)
    df2 = new_weighted_data
    y = data.iloc[:,-1]
    X = new_weighted_data

    forest2 = KNeighborsClassifier()
    cv_results = cross_val_score(forest2,X,y,cv=10)
    forest2.fit(X, y)

    prediction = cross_val_predict(forest2,X,y,cv=10)

    accuracy = metrics.accuracy_score(y, prediction)
    print('Accuracy: ' ,accuracy)

    print(confusion_matrix(y, prediction))
    precision = precision_score(y, prediction)
    print('Precision: %.3f' % precision)
    recall = recall_score(y, prediction, average='binary')
    print('Recall: %.3f' % recall)
    recall_val.append(recall)
    print('\n')

In [ ]:
itr = list(range(num_generations))
zip_iterator = zip(itr, recall_val)
a_dictionary = dict(zip_iterator)
# print(a_dictionary)
res_dict = dict(sorted(a_dictionary.items(), key=lambda item: item[1], reverse = True))

## Sorted Result

In [ ]:
res_dict

# Logistic Regression

## Original Dataset

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

X = data.iloc[:, 0:97]
y = data.iloc[:, -1]

forest = LogisticRegression(random_state=0)
cv_results = cross_val_score(forest,X,y,cv=10)
forest.fit(X, y)

prediction = cross_val_predict(forest,X,y,cv=10)

accuracy = metrics.accuracy_score(y, prediction)
print('Accuracy: ' ,accuracy)

print(confusion_matrix(y, prediction))
precision = precision_score(y, prediction)
print('Precision: %.3f' % precision)
recall = recall_score(y, prediction, average='binary')
print('Recall: %.3f' % recall)
print('\n')

## Transformed Dataset

In [ ]:
recall_val = []
for i in range(num_generations):
    print('Plot number: ', i)
    new_weighted_data = data.iloc[:,0:97].multiply(weight_list[i], axis=1)
    df2 = new_weighted_data
    y = data.iloc[:,-1]
    X = new_weighted_data

    forest2 = LogisticRegression(random_state=0)
    cv_results = cross_val_score(forest2,X,y,cv=10)
    forest2.fit(X, y)

    prediction = cross_val_predict(forest2,X,y,cv=10)

    accuracy = metrics.accuracy_score(y, prediction)
    print('Accuracy: ' ,accuracy)

    print(confusion_matrix(y, prediction))
    precision = precision_score(y, prediction)
    print('Precision: %.3f' % precision)
    recall = recall_score(y, prediction, average='binary')
    print('Recall: %.3f' % recall)
    recall_val.append(recall)
    print('\n')

In [ ]:
itr = list(range(num_generations))
zip_iterator = zip(itr, recall_val)
a_dictionary = dict(zip_iterator)
# print(a_dictionary)
res_dict = dict(sorted(a_dictionary.items(), key=lambda item: item[1], reverse = True))
res_dict

# Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB

X = data.iloc[:, 0:97]
y = data.iloc[:, -1]

forest = GaussianNB()
cv_results = cross_val_score(forest,X,y,cv=10)
forest.fit(X, y)

prediction = cross_val_predict(forest,X,y,cv=10)

accuracy = metrics.accuracy_score(y, prediction)
print('Accuracy: ' ,accuracy)

print(confusion_matrix(y, prediction))
precision = precision_score(y, prediction)
print('Precision: %.3f' % precision)
recall = recall_score(y, prediction, average='binary')
print('Recall: %.3f' % recall)
print('\n')

In [ ]:
recall_val = []
for i in range(num_generations):
    print('Plot number: ', i)
    new_weighted_data = data.iloc[:,0:97].multiply(weight_list[i], axis=1)
    df2 = new_weighted_data
    y = data.iloc[:,-1]
    X = new_weighted_data

    forest2 = GaussianNB()
    cv_results = cross_val_score(forest2,X,y,cv=10)
    forest2.fit(X, y)

    prediction = cross_val_predict(forest2,X,y,cv=10)

    accuracy = metrics.accuracy_score(y, prediction)
    print('Accuracy: ' ,accuracy)

    print(confusion_matrix(y, prediction))
    precision = precision_score(y, prediction)
    print('Precision: %.3f' % precision)
    recall = recall_score(y, prediction, average='binary')
    print('Recall: %.3f' % recall)
    recall_val.append(recall)
    print('\n')

In [ ]:
itr = list(range(num_generations))
zip_iterator = zip(itr, recall_val)
a_dictionary = dict(zip_iterator)
# print(a_dictionary)
res_dict = dict(sorted(a_dictionary.items(), key=lambda item: item[1], reverse = True))
res_dict

In [ ]:
new_weighted_data = data.iloc[:,0:97].multiply(w)
weight_list[0]

In [ ]:
x= new_weighted_data
y=data.iloc[:,-1]
data2 = pd.concat([x, y], ignore_index=False, sort=False, axis=1)

from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0, perplexity=20.0)
tsne_obj= tsne.fit_transform(data2)

In [ ]:
tsne_df1 = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                           'bug':data.iloc[:,-1]})

In [ ]:
sns.scatterplot(x="X", y="Y",hue="bug",data=tsne_df1)

In [ ]:
plt.title("Neo4j Dataset", fontsize=15)
sns.scatterplot(x="X", y="Y",hue="bug",data=tsne_df1,alpha = 0.3)

In [ ]:
x= data.iloc[:, 0:97]
y=data.iloc[:,-1]
data = pd.concat([x, y], ignore_index=False, sort=False, axis=1)

from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0, perplexity=10.0)
tsne_obj= tsne.fit_transform(data)

In [ ]:
tsne_df1 = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                           'bug':data.iloc[:,-1]})

In [ ]:
plt.title("Original Neo4j Dataset", fontsize=15)
sns.scatterplot(x="X", y="Y",hue="bug",data=tsne_df1, alpha = 0.3)